In [1]:
%%capture
%pip install pandas

In [2]:
import pandas as pd
import os
from decimal import Decimal, getcontext, ROUND_HALF_UP

getcontext().prec = 30

In [3]:
files = os.listdir('taxas_juros/raw')

df = pd.DataFrame()
for file in files:
	read_df = pd.read_csv(f'taxas_juros/raw/{file}', sep=';')
	read_df['Classificacao'] = file[:2]
	df = pd.concat([df, read_df])

In [4]:
df.drop('Posicao', axis=1, inplace=True)
df['TaxaJurosAoMes'] = df['TaxaJurosAoMes'].str.replace(',', '.').astype(float)
df['TaxaJurosAoAno'] = df['TaxaJurosAoAno'].str.replace(',', '.').astype(float)

In [5]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO DO BRASIL S.A.             26
BCO BRADESCO S.A.              24
BCO SANTANDER (BRASIL) S.A.    23
CAIXA ECONOMICA FEDERAL        22
BCO SAFRA S.A.                 20
                               ..
BMW FINANCEIRA S.A. - CFI       1
BCO GM S.A.                     1
SOCINAL S.A. CFI                1
BCO MUFG BRASIL S.A.            1
CORA SCFI                       1
Name: count, Length: 171, dtype: int64

In [6]:
df.head()

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoMes,TaxaJurosAoAno,Classificacao
0,Pessoa Jurídica,Capital de giro com prazo superior a 365 dias ...,BCO MERCANTIL DO BRASIL S.A.,0.86,10.86,PJ
1,Pessoa Jurídica,Capital de giro com prazo superior a 365 dias ...,DEUTSCHE BANK S.A.BCO ALEMAO,1.06,13.47,PJ
2,Pessoa Jurídica,Capital de giro com prazo superior a 365 dias ...,BANCO SICOOB S.A.,1.16,14.79,PJ
3,Pessoa Jurídica,Capital de giro com prazo superior a 365 dias ...,BCO HSBC S.A.,1.16,14.84,PJ
4,Pessoa Jurídica,Capital de giro com prazo superior a 365 dias ...,BCO XP S.A.,1.20,15.37,PJ


In [7]:
df['Modalidade'].value_counts()

Modalidade
Crédito pessoal não-consignado - Pré-fixado                                                    89
Cartão de crédito - rotativo total - Pré-fixado                                                65
Cartão de crédito - parcelado - Pré-fixado                                                     62
Cheque especial - Pré-fixado                                                                   60
Desconto de duplicatas - Pré-fixado                                                            53
Crédito pessoal consignado privado - Pré-fixado                                                50
Capital de giro com prazo superior a 365 dias - Pré-fixado                                     48
Crédito pessoal consignado público - Pré-fixado                                                44
Capital de giro com prazo até 365 dias - Pré-fixado                                            42
Aquisição de veículos - Pré-fixado                                                             42
Crédito p

In [8]:
df['SubClassificacao'] = df['Modalidade'].apply(lambda x: x.split(' - ')[-1])
df['Modalidade'] = df['Modalidade'].apply(lambda x: x.rsplit(' - ', 1)[0])

In [9]:
df['Modalidade'].value_counts()

Modalidade
Crédito pessoal não-consignado                    89
Capital de giro com prazo superior a 365 dias     73
Cartão de crédito - rotativo total                65
Cartão de crédito - parcelado                     62
Cheque especial                                   60
Capital de giro com prazo até 365 dias            59
Conta garantida                                   57
Desconto de duplicatas                            53
Crédito pessoal consignado privado                50
Crédito pessoal consignado público                44
Aquisição de veículos                             42
Crédito pessoal consignado INSS                   39
Aquisição de outros bens                          37
Adiantamento sobre contratos de câmbio (ACC)      28
Financiamento imobiliário com taxas de mercado    26
Antecipação de faturas de cartão de crédito       16
Financiamento imobiliário com taxas reguladas     15
Desconto de cheques                               14
Vendor                             

In [10]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      703
Pós-fixado referenciado em juros flutuantes      68
Pós-fixado referenciado em TR                    29
Pós-fixado referenciado em moeda estrangeira     28
Pós-fixado referenciado em IPCA                  10
Name: count, dtype: int64

In [11]:
def transform_taxa(valor_taxa, periodo_desejado, periodo_atual):
	vt = Decimal(valor_taxa)
	pd = Decimal(periodo_desejado)
	pa = Decimal(periodo_atual)
 
	a = 1 + (vt / 100)
	b = pd / pa
	c = a ** b
	d = (c - 1) * 100

	return d

In [12]:
def trim_decimal(n):
	valor = Decimal(str(n).rstrip('0').rstrip('.'))
	return valor.quantize(Decimal('1.0000'), rounding=ROUND_HALF_UP)

In [13]:
df['TaxaJurosAoMes'] = df.apply(lambda x: trim_decimal(transform_taxa(x['TaxaJurosAoAno'], 1, 12)), axis=1)

In [14]:
for instituicao in df.iterrows():
	instituicao = instituicao[1]
	print(f'{instituicao["InstituicaoFinanceira"]}: {instituicao["TaxaJurosAoMes"]}%')

BCO MERCANTIL DO BRASIL S.A.: 0.8629%
DEUTSCHE BANK S.A.BCO ALEMAO: 1.0586%
BANCO SICOOB S.A.: 1.1561%
BCO HSBC S.A.: 1.1598%
BCO XP S.A.: 1.1986%
BCO CATERPILLAR S.A.: 1.2073%
BCO C6 S.A.: 1.3580%
BCO ABC BRASIL S.A.: 1.4182%
BCO FIBRA S.A.: 1.5161%
BCO BS2 S.A.: 1.5204%
BCO DO NORDESTE DO BRASIL S.A.: 1.6326%
BCO DO ESTADO DO RS S.A.: 1.6514%
BANCO RANDON S.A.: 1.6521%
BCO SAFRA S.A.: 1.6737%
ITAÚ UNIBANCO S.A.: 1.6813%
BANCO ORIGINAL: 1.6848%
BCO DAYCOVAL S.A: 1.6945%
BCO DES. DE MG S.A.: 1.7083%
BCO SENFF S.A.: 1.8005%
CARUANA SCFI: 1.8170%
BCO BANESTES S.A.: 1.8586%
BANCO INTER: 1.8980%
BCO INDUSTRIAL DO BRASIL S.A.: 1.9776%
BCO TRIANGULO S.A.: 1.9883%
BCO BRADESCO S.A.: 1.9890%
NOVO BCO CONTINENTAL S.A. - BM: 2.0278%
BCO DO EST. DO PA S.A.: 2.0385%
BCO RNX S.A.: 2.0418%
BCO SOFISA S.A.: 2.0432%
BANCO TOPÁZIO S.A.: 2.0479%
BCO VOLVO BRASIL S.A.: 2.1189%
GAZINCRED S.A. SCFI: 2.1467%
FACTA S.A. CFI: 2.1473%
CAIXA ECONOMICA FEDERAL: 2.1757%
BCO DO BRASIL S.A.: 2.1822%
BCO SANTANDER (